In [20]:
from fuzzywuzzy import process, fuzz
import pandas as pd

def batch_match_anonymized_to_identified(anonymized_df, identified_df, match_columns, threshold=60, 
                                         anonymized_file=None, anonymized_sheet=None):
    """
    Perform batched matching of anonymized rows to identified rows for large datasets.
    
    :param anonymized_df: DataFrame with anonymized data.
    :param identified_df: DataFrame with identified data, including source metadata.
    :param match_columns: List of columns to use for matching.
    :param threshold: Minimum matching score to consider a match valid.
    :param anonymized_file: The file name of the anonymized data.
    :param anonymized_sheet: The sheet name of the anonymized data.
    :return: DataFrame with matched results and metadata for tracking.
    """
    anonymized_df = anonymized_df.copy()
    identified_df = identified_df.copy()
    
    # Create concatenated strings for matching
    anonymized_df['match_key'] = anonymized_df[match_columns].fillna('').apply(lambda x: ' '.join(x), axis=1)
    identified_df['match_key'] = identified_df[match_columns].fillna('').apply(lambda x: ' '.join(x), axis=1)
    
    # Create dictionary for faster lookups
    identified_keys = identified_df['match_key'].to_list()
    identified_lookup = {key: idx for idx, key in enumerate(identified_keys)}
    
    # Initialize results
    matches = []
    for index, row in anonymized_df.iterrows():
        key = row['match_key']
        best_match = process.extractOne(key, identified_keys, scorer=fuzz.token_set_ratio)
        
        if best_match and best_match[1] >= threshold:
            matched_idx = identified_lookup[best_match[0]]
            matched_row = identified_df.iloc[matched_idx]
            matches.append({
                'Anonymized_Index': index,
                'Anonymized_Key': key,
                'Matched_Key': best_match[0],
                'Match_Score': best_match[1],
                'Matched_Row_Index_Excel': matched_row['Original_Row_Index'],  # Use the stored original row index
                'Matched_Row_Details': matched_row.to_dict(),
                'Anonymized_File_Name': anonymized_file,
                'Anonymized_Sheet_Name': anonymized_sheet,
                'Matched_File_Name': matched_row['Source_File_Name'],
                'Matched_Sheet_Name': matched_row['Source_Sheet_Name'],
            })
    
    return pd.DataFrame(matches)

# Load Excel files
file_jan = "11201查處名單.xlsx"
file_feb = "11202查處名單.xlsx"
file_mar = "11203查處名單.xlsx"

# Specify sheets for anonymized and identified data
anonymized_sheets = {"Jan": "表A名單1", "Feb": "表A名單1", "Mar": "表A名單1"}
identified_sheets = {"Jan": "表A名單2", "Feb": "表A名單2", "Mar": "表A名單2"}

# Load identified data with source metadata and original row index
identified_dfs = []
for file_name, sheet_name in zip([file_jan, file_feb, file_mar], 
                                 [identified_sheets["Jan"], identified_sheets["Feb"], identified_sheets["Mar"]]):
    df = pd.read_excel(file_name, sheet_name=sheet_name)
    df['Source_File_Name'] = file_name
    df['Source_Sheet_Name'] = sheet_name
    df['Original_Row_Index'] = df.index + 2  # Add 2 to align with Excel row indexing (1-based, includes header)
    identified_dfs.append(df)

# Combine identified data
combined_identified_data = pd.concat(identified_dfs, ignore_index=True)

# Load anonymized data for January
data_jan_anonymized = pd.read_excel(file_jan, sheet_name=anonymized_sheets["Jan"])

# Perform matching
match_columns = ['地號', '廠址']
results_jan = batch_match_anonymized_to_identified(
    data_jan_anonymized,
    combined_identified_data,
    match_columns,
    threshold=60,
    anonymized_file=file_jan,
    anonymized_sheet=anonymized_sheets["Jan"]
)

# Display results in Jupyter Notebook
results_jan

,Anonymized_Index,Anonymized_Key,Matched_Key,Match_Score,Matched_Row_Index_Excel,Matched_Row_Details,Anonymized_File_Name,Anonymized_Sheet_Name,Matched_File_Name,Matched_Sheet_Name
0,8,大雅區自強段 大林路,大雅區自強段1685地號 大林路211巷21號,61,3,"{'序號': 2, '縣市': '臺中市', '工廠名稱': '無', '市、鎮、村、里':...",11201查處名單.xlsx,表A名單1,11203查處名單.xlsx,表A名單2
1,9,大雅區自強段 大林路,大雅區自強段1685地號 大林路211巷21號,61,3,"{'序號': 2, '縣市': '臺中市', '工廠名稱': '無', '市、鎮、村、里':...",11201查處名單.xlsx,表A名單1,11203查處名單.xlsx,表A名單2
2,57,鹿港鎮東石段 東石段,鹿港鎮東石段2-9地號 東石段2-9地號,77,410,"{'序號': 409, '縣市': '彰化縣', '工廠名稱': '無市招', '市、鎮、村...",11201查處名單.xlsx,表A名單1,11203查處名單.xlsx,表A名單2
3,72,民生路,民生路143號,60,115,"{'序號': 114, '縣市': '新北市', '工廠名稱': '頂隆機械廠', '市、鎮...",11201查處名單.xlsx,表A名單1,11203查處名單.xlsx,表A名單2
4,104,中央路,中央路36號,67,126,"{'序號': 125, '縣市': '新北市', '工廠名稱': '大漢食品企業社', '市...",11201查處名單.xlsx,表A名單1,11203查處名單.xlsx,表A名單2
5,105,光復路,光復路335號,60,202,"{'序號': 201, '縣市': '臺中市', '工廠名稱': '綠野奇木藝品社', '市...",11201查處名單.xlsx,表A名單1,11203查處名單.xlsx,表A名單2
6,127,復興路,復興路402號,60,18,"{'序號': 17, '縣市': '新北市', '工廠名稱': '北貿紙品有限公司', '市...",11201查處名單.xlsx,表A名單1,11203查處名單.xlsx,表A名單2
7,136,壟鉤路,壟鉤路1之8號,60,89,"{'序號': 88, '縣市': '新北市', '工廠名稱': '文炬工業股份有限公司', ...",11201查處名單.xlsx,表A名單1,11203查處名單.xlsx,表A名單2
8,164,中正路,中正路139號,60,311,"{'序號': 310, '縣市': '臺南市', '工廠名稱': '台南市玉井區農會', '...",11201查處名單.xlsx,表A名單1,11203查處名單.xlsx,表A名單2
9,168,俊英街,俊英街1之1號,60,117,"{'序號': 116, '縣市': '新北市', '工廠名稱': '天香豆腐工廠', '市、...",11201查處名單.xlsx,表A名單1,11203查處名單.xlsx,表A名單2
